In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
import sys
import os

basedrive = "drive/My Drive/upper limb use assessment"
os.chdir(basedrive)
sys.path.append("scripts")

In [3]:
!pip install ahrs

     |████████████████████████████████| 197 kB 23.4 MB/s 


In [4]:
!pip install scikit-learn==0.23

     |████████████████████████████████| 7.3 MB 13.6 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.0 which is incompatible.


In [5]:
import classification_algorithms as ca
import reduced_models as rm
import task_analysis as ta

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.neighbors.kde module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Comparison

In [ ]:
ca.generate_activity_counts_output('control')

Generating activity counts...Done 


In [ ]:
ca.generate_vector_magnitude_output('control')

Generating vector magnitude output...Done 


In [ ]:
ca.generate_gm_score_output('patient')

Generating GM scores...Done 


In [ ]:
ca.generate_hybrid_gmac_output('control')

Generating modified GM scores...Done 


In [ ]:
ca.generate_and_save_features('control')

Generating features (4hz) for machine learning...Done 


In [ ]:
ca.generate_intersubject_output('control', 'rf')

Generating inter-subject output...
Left/Affected arm (10 subjects):


100%|██████████| 10/10 [4:00:49<00:00, 1444.97s/it]


Right/Unaffected arm (10 subjects:


100%|██████████| 10/10 [4:03:31<00:00, 1461.15s/it]


In [ ]:
ca.generate_intersubject_output('control', 'svm')

Generating inter-subject output...
Left/Affected arm (5 subjects):


100%|██████████| 5/5 [05:04<00:00, 60.81s/it]


Right/Unaffected arm (5 subjects:


100%|██████████| 5/5 [06:03<00:00, 72.69s/it]


In [ ]:
ca.generate_intersubject_output('patient', 'mlp')

Generating inter-subject output...
Left/Affected arm (5 subjects):


TypeError: ignored

# Reduced models

In [ ]:
rm.generate_intrasubject_output('patient', 'ax')

Generating intra-subject output...
Iterating 10 times:


100%|██████████| 10/10 [49:53<00:00, 299.37s/it]


# Task analysis

In [7]:
ta.generate_intersubject_output('unaffected', 'openbottle')

scripts/classification_algorithms.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['anorm'] = [np.linalg.norm(acc_vectors[:, i]) for i in range(len(data))]


In [7]:
hand = 'left'
task = 'walk'

if hand == 'right':
    subject_type = 'control'
    _, data = ca.read_data(subject_type)
elif hand == 'left':
    subject_type = 'control'
    data, _ = ca.read_data(subject_type)
elif hand == 'affected':
    subject_type = 'patient'
    data, _ = ca.read_data(subject_type)
elif hand == 'unaffected':
    subject_type = 'patient'
    _, data = ca.read_data(subject_type)
else:
    raise Exception(f"Invalid hand: {hand}. Use 'left', 'right', 'affected' or 'unaffected' instead.")

if task == 'walk':
    remove_tasks = ['Walk25', 'OnSwitch', 'OpenDoor']
elif task == 'openbottle':
    remove_tasks = ['OpenBottle']
elif task == 'drinkcup':
    remove_tasks = ['DrinkCup']
else:
    raise Exception(f"Invalid task: {task}. Use 'walk', 'openbottle' or 'drinkcup' instead.")

feat = ca.compute_features(data[(data['task'] != ' ')], 4)
tgt = 'l' if (hand == 'left' or hand == 'affected') else 'r'

# rfl1 = ta.train_validate_and_test_intersubject(feat, remove_tasks=remove_tasks, remove_from=None).rename(
#     columns={'pred': tgt})[[tgt]]
# rfl2 = train_validate_and_test_intersubject(feat, remove_tasks=remove_tasks, remove_from='test').rename(columns={'pred': tgt})[[tgt]]
# rfl3 = train_validate_and_test_intersubject(feat, remove_tasks=remove_tasks, remove_from='train').rename(columns={'pred': tgt})[[tgt]]
rfl4 = ta.train_validate_and_test_intersubject(feat, remove_tasks=remove_tasks, remove_from='both').rename(columns={'pred': tgt})[[tgt]]

# rfl1.to_csv(subject_type + f'/classifier outputs/rf{tgt}_inter_{task}.csv')
# rfl2.to_csv(subject_type + f'/classifier outputs/rf{tgt}_inter_test_without_{task}.csv')
# rfl3.to_csv(subject_type + f'/classifier outputs/rf{tgt}_inter_train_without_{task}.csv')
rfl4.to_csv(subject_type + f'/classifier outputs/rf{tgt}_inter_no_{task}.csv')

scripts/classification_algorithms.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['anorm'] = [np.linalg.norm(acc_vectors[:, i]) for i in range(len(data))]
